In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    # "danjacobellis/LSDIR_WEBP_q20",
    # "danjacobellis/LSDIR_DGML_q6",
    "danjacobellis/LSDIR_RGB_Li_48c_J3_nf8",
    "danjacobellis/LSDIR_stable_diffusion_3_fp16",
    "danjacobellis/LSDIR_RGB_Li_12c_J3_nf8",
]

dimensionality_reduction = [
    # 1,
    # 4,
    4,
    12,
    16,
]

names = [
    # 'WEBP',
    'Cheng2020',
    'WaLLoC',
    'Stable Diffusion 3',
    'WaLLoC',
]

metrics = [
    'PSNR',
    'MSSIM',
    'LPIPS_dB',
    'DISTS_dB',
]

columns = [
    'Name',
    'Dimensionality Reduction',
    'Compression Ratio',
    'Encode (CPU)',
    'Decode (CPU)',
    'Encode (GPU)',
    'Decode (GPU)',
] + metrics

In [3]:
data_rows = []
for i_method, method in enumerate(methods):
    ds = load_dataset(method, split='validation')
    row = [names[i_method], dimensionality_reduction[i_method]]
    compression_ratio = 24 / np.mean(ds['bpp'])
    encode_MP_per_second = np.mean(np.array(ds['n_pixels'])/np.array(ds['cpu_encode_time'])/1e6)
    decode_MP_per_second = np.mean(np.array(ds['n_pixels'])/np.array(ds['cpu_decode_time'])/1e6)
    encode_MP_per_second_GPU = np.mean(np.array(ds['n_pixels'])/np.array(ds['encode_time'])/1e6)
    decode_MP_per_second_GPU = np.mean(np.array(ds['n_pixels'])/np.array(ds['decode_time'])/1e6)
    row.append(compression_ratio)
    row.append(encode_MP_per_second)
    row.append(decode_MP_per_second)
    row.append(encode_MP_per_second_GPU)
    row.append(decode_MP_per_second_GPU)
    for metric in metrics:
        mean = np.mean(ds[metric])
        row.append(mean)
    data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

df_rounded = df.round(4)
md_table = df_rounded.to_markdown(index=False)
display(Markdown(md_table))

| Name               |   Dimensionality Reduction |   Compression Ratio |   Encode (CPU) |   Decode (CPU) |   Encode (GPU) |   Decode (GPU) |    PSNR |   MSSIM |   LPIPS_dB |   DISTS_dB |
|:-------------------|---------------------------:|--------------------:|---------------:|---------------:|---------------:|---------------:|--------:|--------:|-----------:|-----------:|
| Cheng2020          |                          4 |              8.5313 |        13.9753 |         0.47   |        18.7225 |        130.437 | 33.5474 |  0.9902 |    11.2465 |    19.2875 |
| WaLLoC             |                         12 |              6      |         0.1949 |         0.1013 |         6.9705 |        332.164 | 20.8806 |  0.8399 |     8.3253 |    13.8466 |
| Stable Diffusion 3 |                         16 |             35.1925 |        22.0586 |         0.4657 |        36.3303 |        211.692 | 27.4524 |  0.97   |     6.5131 |    13.8514 |